# Quantum State Tomography

In [1]:
import qiskit
from qiskit_experiments.framework import ParallelExperiment
from qiskit_experiments.library import StateTomography

# For simulation
from qiskit.providers.aer import AerSimulator
from qiskit.test.mock import FakeParis

# Noisy simulator backend
backend = AerSimulator.from_backend(FakeParis())

## State Tomography Experiment

To run a state tomography experiment we initialize the experiment with a circuit to prepare the state to be measured. We can also pass in an `Operator`, or a `Statevector` to describe the preparation circuit.

In [2]:
# Run experiments

# GHZ State preparation circuit
nq = 2
qc_ghz = qiskit.QuantumCircuit(nq)
qc_ghz.h(0)
qc_ghz.s(0)
for i in range(1, nq):
    qc_ghz.cx(0, i)

# QST Experiment
qstexp1 = StateTomography(qc_ghz)
qstdata1 = qstexp1.run(backend, seed_simulation=100).block_for_results()
qstresult = qstdata1.analysis_results(0)

print('FIT RESULT')
print(qstresult)

FIT RESULT

Analysis Result: StateTomography
Analysis Result ID: 95c2123e-8164-4fe2-a7c0-bbc1ccf04328
Experiment ID: a903a1ad-4535-4c28-b7c8-db5f9aec54ae
Device Components: [<Qubit(Q0)>, <Qubit(Q1)>]
Quality: None
Verified: False
Result Data:
- state: DensityMatrix([[ 0.47672008+0.00000000e+00j,  0.01743136+3.38352041e-03j,
                 0.02372273-1.18893984e-02j,  0.00085292-4.48489363e-01j],
               [ 0.01743136-3.38352041e-03j,  0.02290845+0.00000000e+00j,
                 0.00348649+5.07348117e-03j,  0.00482618+1.28395469e-03j],
               [ 0.02372273+1.18893984e-02j,  0.00348649-5.07348117e-03j,
                 0.03314588+0.00000000e+00j, -0.01239848-2.14036205e-02j],
               [ 0.00085292+4.48489363e-01j,  0.00482618-1.28395469e-03j,
                -0.01239848+2.14036205e-02j,  0.46722559-2.71050543e-20j]],
              dims=(2, 2))
- fitter_metadata: {'fitter': 'linear_inversion', 'fitter_time': 0.002125263214111328}
- success: True
- state_metadata: {'e

### Tomography Results

The main results for tomography are the fitted state, which is stored in the `"state"` key as a `DensityMatrix` object:

In [3]:
print(qstresult.data()["state"])

DensityMatrix([[ 0.47672008+0.00000000e+00j,  0.01743136+3.38352041e-03j,
                 0.02372273-1.18893984e-02j,  0.00085292-4.48489363e-01j],
               [ 0.01743136-3.38352041e-03j,  0.02290845+0.00000000e+00j,
                 0.00348649+5.07348117e-03j,  0.00482618+1.28395469e-03j],
               [ 0.02372273+1.18893984e-02j,  0.00348649-5.07348117e-03j,
                 0.03314588+0.00000000e+00j, -0.01239848-2.14036205e-02j],
               [ 0.00085292+4.48489363e-01j,  0.00482618-1.28395469e-03j,
                -0.01239848+2.14036205e-02j,  0.46722559-2.71050543e-20j]],
              dims=(2, 2))


The state fidelity of the fitted state with the ideal state prepared by the input circuit is stored in the `"state_fidelity"` result field. Note that if the input circuit contained any measurements the ideal state cannot be automatically generated and this field will be set to `None`.

In [4]:
print("State Fidelity = {:.5f}".format(qstresult.data()["state_fidelity"]))

State Fidelity = 0.92046


#### Additional state metadata

Additional data is stored in the tomography under the `"state_metadata"` field. This includes
- `eigvals`: the eigenvalues of the fitted state
- `trace`: the trace of the fitted state
- `positive`: Whether the eigenvalues are all non-negative
- `positive_delta`: the deviation from positivity given by 1-norm of negative eigenvalues.

If trace rescaling was performed this dictionary will also contain a `raw_trace` field containing the trace before rescaling.
Futhermore, if the state was rescaled to be positive or trace 1 an additional field `raw_eigvals` will contain the state eigenvalues before rescaling was performed.

In [5]:
pprint(qstresult.data()["state_metadata"])

Pretty printing has been turned OFF


To see the effect of rescaling we can perform a "bad" fit with very low counts

In [6]:
# QST Experiment
bad_data = qstexp1.run(backend, shots=10, seed_simulation=100).block_for_results()
bad_result = bad_data.analysis_results(0)

print(bad_result.data()["state_fidelity"])
for key, val in bad_result.data()["state_metadata"].items():
    print(f'- {key}: {val}')

0.8686814149289035
- eigvals: [0.92074124 0.07925876 0.         0.        ]
- raw_eigvals: [ 1.00806592  0.16658344  0.07029948 -0.24494884]
- trace: 1.0000000000000049
- positive: True
- positive_delta: 0.0


## Tomography Fitters

The default fitters is `linear_inversion`, which reconstructs the state using *dual basis* of the tomography basis. This will typically result in a non-postive reconstructed state. This state is rescaled to be postive-semidfinite (PSD) by computing its eigen-decomposition and rescaling its eigenvalues using the approach from *J Smolin, JM Gambetta, G Smith, Phys. Rev. Lett. 108, 070502 (2012), [open access](https://arxiv.org/abs/arXiv:1106.5458).

There are several other fitters are included (See API documentation for details). For example if `cvxpy` is installed we can use the `cvxpy_gaussian_lstsq` fitter which allows constraining the fit to be PSD without requiring rescaling.

In [7]:
qstexp1.run_analysis(qstdata1, fitter='cvxpy_linear_lstsq')
qstresult2 = qstdata1.analysis_results(-1)

print('FIT RESULT')
print(qstresult2)

FIT RESULT

Analysis Result: StateTomography
Analysis Result ID: ce355bf5-8b51-401f-ac79-786cbb28c266
Experiment ID: a903a1ad-4535-4c28-b7c8-db5f9aec54ae
Device Components: [<Qubit(Q0)>, <Qubit(Q1)>]
Quality: None
Verified: False
Result Data:
- state: DensityMatrix([[ 0.47635077+0.00000000e+00j,  0.01748862+3.89240856e-03j,
                 0.02422926-1.18500323e-02j,  0.0008573 -4.47804563e-01j],
               [ 0.01748862-3.89240856e-03j,  0.02291869+0.00000000e+00j,
                 0.00348893+5.05709989e-03j,  0.00532333+1.37176137e-03j],
               [ 0.02422926+1.18500323e-02j,  0.00348893-5.05709989e-03j,
                 0.03382784-8.67361738e-19j, -0.01236219-2.08270256e-02j],
               [ 0.0008573 +4.47804563e-01j,  0.00532333-1.37176137e-03j,
                -0.01236219+2.08270256e-02j,  0.46690269+2.71050543e-20j]],
              dims=(2, 2))
- fitter_metadata: {'cvxpy_solver': 'CVXOPT', 'cvxpy_status': 'optimal', 'fitter': 'cvxpy_linear_lstsq', 'fitter_time': 0.04

## Parallel Tomography Experiment

We can also use the `qiskit_experiments.ParallelExperiment` class to run subsystem tomography on multiple qubits in parallel.

For example if we want to perform 1-qubit QST on several qubits at once:

In [8]:
from math import pi
num_qubits = 5
gates = [qiskit.circuit.library.RXGate(i * pi / (num_qubits - 1))
         for i in range(num_qubits)]

subexps = [
    StateTomography(gate, qubits=[i])
    for i, gate in enumerate(gates)
]
parexp = ParallelExperiment(subexps)
pardata = parexp.run(backend, seed_simulation=100).block_for_results()
print(pardata.analysis_results(-1))


Analysis Result: ParallelExperiment
Analysis Result ID: 04a4fb78-fbf5-4705-9032-77b81a85b019
Experiment ID: b761f6c3-f280-4cbe-bfc5-a9a78270697c
Device Components: [<Qubit(Q0)>, <Qubit(Q1)>, <Qubit(Q2)>, <Qubit(Q3)>, <Qubit(Q4)>]
Quality: None
Verified: False
Result Data:
- experiment_types: ['StateTomography', 'StateTomography', 'StateTomography', 'StateTomography', 'StateTomography']
- experiment_ids: ['d79b01b2-d62a-4eed-ae57-efdab09f46d7', 'e4a8c99b-62bd-45c4-8506-8c28a5a91b2a', '451a1368-15c1-4194-b3b4-c7c0a0e5cc23', '81636e00-bbef-4ca1-802d-c1b55f414110', '80dd68ce-390e-479f-8542-07fe6c21427a']
- experiment_qubits: [(0,), (1,), (2,), (3,), (4,)]
- success: True


View component experiment analysis results

In [9]:
for i in range(parexp.num_experiments):
    expdata = pardata.component_experiment_data(i)
    result = expdata.analysis_results(-1)
    
    print(f'\nPARALLEL EXP {i}: FIT RESULT')
    print(result)


PARALLEL EXP 0: FIT RESULT

Analysis Result: StateTomography
Analysis Result ID: 5db46640-4a02-4c78-aa82-d0cb8be8e4dc
Experiment ID: d79b01b2-d62a-4eed-ae57-efdab09f46d7
Device Components: [<Qubit(Q0)>]
Quality: None
Verified: False
Result Data:
- state: DensityMatrix([[0.98144531+0.j        , 0.00878906-0.03417969j],
               [0.00878906+0.03417969j, 0.01855469+0.j        ]],
              dims=(2,))
- fitter_metadata: {'fitter': 'linear_inversion', 'fitter_time': 0.00010895729064941406}
- success: True
- state_metadata: {'eigvals': array([0.98273708, 0.01726292]), 'trace': 1.0000000000000002, 'positive': True, 'positive_delta': 0.0}
- state_fidelity: 0.9814453125

PARALLEL EXP 1: FIT RESULT

Analysis Result: StateTomography
Analysis Result ID: 911eeebb-139e-42d8-b9ba-785cc49fbac0
Experiment ID: e4a8c99b-62bd-45c4-8506-8c28a5a91b2a
Device Components: [<Qubit(Q1)>]
Quality: None
Verified: False
Result Data:
- state: DensityMatrix([[0.85644531+0.j        , 0.015625  +0.33496094j]